In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, BatchNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd
import cv2

drive_dir = '/content/drive/My Drive/'
data_dir = drive_dir + 'OCT/'
pre_dir = data_dir + 'proc/'
rnfl_dir = data_dir + 'RNFLT/'
scan_dir = data_dir + 'Bscan/'

data = pd.read_csv(data_dir + 'ReadMe/data_summary.csv')

imgs, labels, count = [], [], 0
for fn in sorted(os.listdir(pre_dir)):
  if count < 600:
    print(fn)
    for j in range(200):
      labels.append(data['glaucoma'][count])

    img = np.load(pre_dir + fn)
    #img = img['images']
    img = img['bscans']

    for im in img:
      imgs.append(im)

  count += 1

print(len(imgs), len(labels))
imgs = np.array(imgs)
labels = np.array(labels)

In [ ]:
from keras.applications import EfficientNetB0

image_size = (200, 300)
batch_size = 32
epochs = 10
num_classes = 1

enb = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(200, 300, 3))

enb.trainable = True

single = Input(shape=(200, 300, 1))
x = Conv2D(3, (1, 1), padding='same')(single)
enb_output = enb(x)

gap = GlobalAveragePooling2D()(enb_output)
final = Dense(1, activation='sigmoid')(gap)

model = Model(inputs=single, outputs=final)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(),
                tf.keras.metrics.Recall()])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

filepath = "/content/drive/MyDrive/OCT/models/efficientnet/glauc-epoch-{epoch:02d}.hdf5"

cb = [
    ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max', save_freq='epoch')
]

X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

imgs, labels = None, None

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[cb])

In [ ]:
model.history.history
keys = list(model.history.history.keys())
keys

In [ ]:
train_loss = model.history.history[keys[0]]
print(keys[0])

val_loss = model.history.history[keys[5]]
print(keys[5])

train_accuracy = model.history.history[keys[1]]
print(keys[1])

val_accuracy = model.history.history[keys[6]]
print(keys[6])

train_auc = model.history.history[keys[2]]
print(keys[2])

val_auc = model.history.history[keys[7]]
print(keys[7])

train_precision = model.history.history[keys[3]]
print(keys[3])

val_precision = model.history.history[keys[8]]
print(keys[8])

train_recall = model.history.history[keys[4]]
print(keys[4])

val_recall = model.history.history[keys[9]]
print(keys[9])

epochs = range(1, len(train_loss) + 1)

In [ ]:
import matplotlib.pyplot as plt

results_dir = '/content/drive/MyDrive/OCT/models/efficient_net_new/results/'

plt.plot(epochs, train_loss, label='Train')
plt.plot(epochs, val_loss, label='Validation')
plt.title('Loss over time')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(results_dir + 'loss.png')
plt.show()

plt.plot(epochs, train_accuracy, label='Train')
plt.plot(epochs, val_accuracy, label='Validation')
plt.title('Accuracy over time')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(results_dir + 'accuracy.png')
plt.ylim(0, 1)
plt.show()

plt.plot(epochs, train_auc, label='Traini')
plt.plot(epochs, val_auc, label='Validation')
plt.title('AUC over time')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()
plt.savefig(results_dir + 'auc.png')
plt.ylim(0, 1)
plt.show()

plt.plot(epochs, train_precision, label='Train')
plt.plot(epochs, val_precision, label='Validation')
plt.title('Precision over time')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.savefig(results_dir + 'precision.png')
plt.ylim(0, 1)
plt.show()

plt.plot(epochs, train_recall, label='Train')
plt.plot(epochs, val_recall, label='Validation')
plt.title('Recall over time')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.savefig(results_dir + 'recall.png')
plt.ylim(0, 1)
plt.show()